# データの加工

In [ ]:
# データのダウンロード
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() # x：画像，y：ラベル

In [ ]:
# テストデータの1つを事前に取り出しておく
x_t, y_t  = x_test[42], y_test[42]

In [ ]:
x_train[0].shape # 28×28 pixelの画像

In [ ]:
# trainの一番最初の画像
import matplotlib.pyplot as plt
plt.imshow(x_train[0])

# trainの一番最初のラベル
print("ラベル：",y_train[0])

In [ ]:
# validationに分割
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=1/6)

In [ ]:
# データをシャッフル
import numpy as np
train_idx = np.random.permutation(len(x_train))
valid_idx = np.random.permutation(len(x_valid))

x_train, y_train = x_train[train_idx], y_train[train_idx]
x_valid, y_valid = x_valid[valid_idx], y_valid[valid_idx]

In [ ]:
print("train：{}枚".format(x_train.shape[0]))
print("valid：{}枚".format(x_valid.shape[0]))
print("test ：{}枚".format(x_test.shape[0]))

In [ ]:
# 学習のために変形する(チャンネルに当たる次元を拡張している)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_valid = x_valid.reshape(x_valid.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [ ]:
# 0-255の整数値を0〜1の小数に変換する（画像の標準化）
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_valid /= 255
x_test /= 255

In [ ]:
from tensorflow.keras import utils
# one-hot vector形式に変換する
y_train = utils.to_categorical(y_train, 10)
y_valid = utils.to_categorical(y_valid, 10)
y_test = utils.to_categorical(y_test, 10)

In [ ]:
print("one-hot vector 処理前：{}".format(y_t))
print("one-hot vector 処理前：{}".format(y_test[42]))

# モデル

In [ ]:
# モデルを作る
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1))) # 入力の形が必要
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(
    model,
    show_shapes=True,
)

# 学習

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_valid, y_valid))

# 評価

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# 使用

In [ ]:
# テストデータを取り出して適用してみる
plt.imshow(x_t)

# trainの一番最初のラベル
print("ラベル：",y_t)

In [ ]:
print("推測したラベル：" ,np.argmax(model.predict(x_test[42].reshape(1,28,28,1),
                                    batch_size=None, verbose=0, steps=None)))